<a href="https://www.kaggle.com/code/echowuzangye/adversarial-validation?scriptVersionId=148901884" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('../input/porto-seguro-safe-driver-prediction/train.csv')
test = pd.read_csv('../input/porto-seguro-safe-driver-prediction/test.csv')

In [ ]:
print(train.shape, test.shape)

1. Run adversarial validation test with RandomForestClassifier model, see if the training and test data most probably originated from the same data distribution

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import roc_auc_score


Drop the target and also any other identifiers; when the identifier is progressive, the adversarial validation may return a high ROC-AUC score

In [ ]:
train = train.drop(["id", "target"], axis=1)
test = test.drop(["id"], axis=1)
X = pd.concat([test,train])
y = [0] * len(train) + [1] * len(test)

Generate RandomForestClassifier predictions for the data using the cross_val_predict function, which automatically creates a cross-validation scheme and stores the predictions on the validation fold:

In [ ]:
model = RandomForestClassifier()
cv_preds = cross_val_predict(model, X_rf, y_rf, cv=5, n_jobs=-1, method='predict_proba')

This session took around 1h

In [ ]:
print(roc_auc_score(y_true=y, y_score=cv_preds[:,1]))
#  0.5003436561938691

AUC-ROC is around 0.5; We cannot easily distinguish test data from train data, they are from the same distribution.
https://developers.google.com/machine-learning/crash-course/classification/roc-and-auc

2. Now try with XGBoost.

In [ ]:
import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import plot_importance
from xgboost import cv
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 12})

In [ ]:
# create DMatrix (the XGBoost data structure)
XGBdata = xgb.DMatrix(data=X,label=y)

In [ ]:
# set XGBoost parameters
params = {"objective":"binary:logistic",
          "eval_metric":"logloss",
          'learning_rate': 0.05,
          'max_depth': 5, }

In [ ]:
# perform cross validation with XGBoost
cross_val_results = cv(dtrain=XGBdata, params=params, 
                       nfold=5, metrics="auc", 
                       num_boost_round=200,early_stopping_rounds=20,
                       as_pandas=True)

This run only took 5 minutes

In [ ]:
cross_val_results

In [ ]:
# print out the final result
print((cross_val_results["test-auc-mean"]).tail(1))

* Approach 1 was based on Chapter 1 exercise 1 of https://www.amazon.com/Kaggle-Workbook-Self-learning-exercises-competitions/dp/1804611212
* Approach 2 was modified from https://www.kaggle.com/code/carlmcbrideellis/what-is-adversarial-validation